In [2]:
def show_matrix(matrix, name='ans', decimal_places=2, scientific_notation=True):
  """
  Displays a matrix with the specified number of decimal places.

  Parameters:
  ---
  - matrix: numpy.ndarray, the matrix to be displayed.
  - name: str, the name to be used in the display header (default is 'ans').
  - decimal_places: int, the number of decimal places to display (default is 2).
  - scientific_notation: bool, if True, uses scientific notation; otherwise, uses fixed-point notation (default is True).
  """
  # Define the format pattern based on whether scientific notation is used
  pattern = f"{{:.{decimal_places}{'e' if scientific_notation else 'f'}}}"

  def format_elem(elem):
    """
    Formats a single matrix element according to the defined pattern.

    Parameters:
    - elem: the element to be formatted.

    Returns:
    - str: the formatted element.
    """
    return pattern.format(elem)

  # Calculate the maximum width required for each column
  col_widths = [max(map(len, map(format_elem, col))) for col in matrix.T]

  print(f"{name} =")  # Print the matrix name
  # Calculate the spacing for the matrix border
  nspaces = sum(col_widths) + 2 * matrix.shape[1]

  # Print the top border of the matrix
  print("    ┌" + " " * nspaces + "┐")

  # Print each row of the matrix
  for row in matrix:
    # Format each element of the row and right-align according to column width
    formatted_row = "  ".join(format_elem(e).rjust(w)
                              for e, w in zip(row, col_widths))
    print(f"    │ {formatted_row} │")

  # Print the bottom border of the matrix
  print("    └" + " " * nspaces + "┘\n")

In [3]:
import numpy as np


def proj(v, u):
  """
  Retorna o projeto do vetor v sobre o vetor u.
  """
  return (v @ u) / (u @ u) * u


def gram_schmidt(vectors):
  """
  Algoritmo de Gram-Schmidt para ortogonalizar um conjunto de vetores.

  Parameters:
      vectors (np.ndarray): Conjunto de vetores a serem ortogonalizados.

  Returns:
      np.ndarray: Conjunto de vetores ortogonais.
  """
  u = np.copy(vectors)  # Cria uma cópia dos vetores
  for i in range(len(vectors)):
    for j in range(i):
      u[i] -= proj(vectors[i], u[j])  # Subtrai a projeção
  return u


def normalize(vectors):
  """
  Normaliza um conjunto de vetores. Caso o vetor seja nulo, ele é retornado sem alterações.

  Parameters:
      vectors (np.ndarray): Conjunto de vetores a serem normalizados.

  Returns:
      np.ndarray: Vetores normalizados.
  """
  norms = np.linalg.norm(
      vectors, axis=1)  # Calcula as normas de todos os vetores
  # Normaliza os vetores, substituindo os nulos por si mesmos
  return np.divide(vectors.T, norms, where=norms != 0).T


# Exemplo de uso
vectors = np.array([[12., -51., 4.],
                    [6., 167., -68.],
                    [-4., 24., -41]])

ortho_basis = gram_schmidt(vectors)
ortonormal_basis = normalize(ortho_basis)

# Verificando ortonormalidade
print("Base Ortonormal:")
for i, u in enumerate(ortonormal_basis):
  print(f'u[{i}] = ', np.round(u, 2))

# Verifica a ortonormalidade, o produto interno deve ser a matriz identidade
print('\n', np.round(ortonormal_basis.T @ ortonormal_basis, 2))

Base Ortonormal:
u[0] =  [ 0.23 -0.97  0.08]
u[1] =  [ 0.62  0.08 -0.78]
u[2] =  [-0.75 -0.23 -0.62]

 [[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0. -0.  1.]]


In [28]:
import numpy as np


def gram_schmidt_modified(vectors):
  """
  Aplica o processo de Gram-Schmidt modificado para ortogonalizar um conjunto de vetores
  e retorna a matriz Q ortonormal.

  Parâmetros:
      vectors (array-like): Conjunto de vetores a ser ortogonalizado (forma (m, n)).

  Retorna:
      Q (ndarray): Matriz ortonormal resultante após a ortogonalização.
  """
  A = np.array(
      vectors, dtype=float)  # Garante que os vetores são do tipo float
  m, n = A.shape
  Q = np.empty((m, n), dtype=float)  # Inicializa Q com o formato correto

  for i in range(n):
    # Inicializa o vetor Q[:, i] com o vetor A[:, i]
    q_i = A[:, i]

    for j in range(i):
      # Projeta o vetor A[:, i] sobre Q[:, j] e subtrai essa projeção
      q_j = Q[:, j]
      # Usando o operador @ para o produto escalar
      q_i -= (q_j @ A[:, i]) * q_j

    # Normaliza o vetor resultante
    norm_q_i = np.linalg.norm(q_i)
    if norm_q_i > 0:
      Q[:, i] = q_i / norm_q_i
    else:
      # Caso a norma seja zero, preserva o vetor original (tratamento de erro)
      Q[:, i] = q_i

  return Q


# Exemplo de uso
A = np.array([[0., 1., 2.],
              [2., 0., 0.],
              [1., 1., 1.]])

# Calculando apenas a matriz Q (Ortonormal)
Q = gram_schmidt_modified(A)

# Verificando a matriz Q
print("Matriz Q (Ortonormal):")
print(np.round(Q, 2))

# Verificando se Q é ortonormal (Q^T * Q deve ser a identidade)
print("\nVerificando se Q^T * Q é a identidade:")
# Usando o operador @ para multiplicação de matrizes
print(np.round(Q.T @ Q, 2))

Matriz Q (Ortonormal):
[[ 0.    1.    0.  ]
 [ 0.45  0.    0.89]
 [ 0.89  0.   -0.45]]

Verificando se Q^T * Q é a identidade:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [36]:
import numpy as np


def gram_schmidt_modified(vectors):
  """
  Aplica o processo de Gram-Schmidt modificado para ortogonalizar um conjunto de vetores
  e calcular a fatoração QR de uma matriz.

  Parâmetros:
      vectors (array-like): Conjunto de vetores a ser ortogonalizado (forma (m, n)).

  Retorna:
      Q (ndarray): Matriz ortonormal (m, n).
      R (ndarray): Matriz triangular superior (n, n).
  """
  A = np.array(vectors, dtype=float)  # Converte os vetores para float
  m, n = A.shape
  Q = np.zeros((m, n), dtype=float)  # Inicializa Q com zeros
  R = np.zeros((n, n), dtype=float)  # Inicializa R com zeros

  for i in range(n):
    # Inicializa o vetor Q[:, i] com o vetor A[:, i]
    q_i = A[:, i]

    for j in range(i):
      # Calcula os coeficientes de projeção (R_ij)
      R[j, i] = Q[:, j] @ A[:, i]
      q_i -= R[j, i] * Q[:, j]  # Subtrai a projeção do vetor Q[:, i]

    # Normaliza o vetor Q[:, i] e calcula o valor R[i, i]
    norm_q_i = np.linalg.norm(q_i)
    if norm_q_i > 1e-10:  # Evita a divisão por zero
      Q[:, i] = q_i / norm_q_i
      R[i, i] = norm_q_i
    else:
      Q[:, i] = q_i  # Preserva o vetor caso sua norma seja zero
      R[i, i] = 0  # Caso o vetor tenha norma zero, R[i, i] é zero

  return Q, R


# Exemplo de uso
A = np.array([[1., 2., 3.],
              [4., 5., 6.],
              [7., -3., -2.]])

# Calculando a fatoração QR
Q, R = gram_schmidt_modified(A)

# Verificando a fatoração QR
print("Matriz Q (Ortonormal):")
print(np.round(Q, 2))
print("\nMatriz R (Triangular Superior):")
print(np.round(R, 2))

# Verificando se A = QR
print("\nVerificando se A = QR:")
A_reconstructed = Q @ R
print(np.round(A_reconstructed, 2))

Matriz Q (Ortonormal):
[[ 0.12  0.32  0.94]
 [ 0.49  0.8  -0.34]
 [ 0.86 -0.5   0.06]]

Matriz R (Triangular Superior):
[[8.12 0.12 1.6 ]
 [0.   6.16 6.78]
 [0.   0.   0.66]]

Verificando se A = QR:
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7. -3. -2.]]


In [37]:
b = np.array([[14.], [32.], [-5.]])
print(np.round(Q.T @ b, 2))

[[13.17]
 [32.67]
 [ 1.98]]


In [12]:
U = np.array([[1., 1., 0., 1.],
             [-1., 1., 1., 0.],
             [1., 1., 1., 1.]])

b = np.array([[2], [1], [0], [2]])
print(U @ b)

[[ 5.]
 [-1.]
 [ 5.]]
